## Anomoly Detection - AMF Handover Duration
### 5G Simultor & Anamoly Generation

#### Conditions created
generate_anomaly_periods(): Creates 2-3 random time periods where anomalies will occur.
is_in_anomaly_period(): Checks if the current time is within an anomaly period.
introduce_anomaly(): During anomaly periods, introduces high latency (1-3 seconds) 70% of the time, and very high latency (3-5 seconds) 20% of the time.


In [ ]:
!git clone https://github.com/drcoopertbbt/open-digital-platform-2_0.git

### AMF Handover Data - Prometheus Capture
- **"Cumulative_Sum_Value"** shows total time (in seconds) spent on handover operations since the metric collection began

In [ ]:
import pandas as pd

# Load the CSV file
file_path = 'open-digital-platform-2_0/5G_Emulator_API/core_network/amf-handover-request-durations.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
df.head()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load the image from the file path
file_path = 'open-digital-platform-2_0/5G_Emulator_API/core_network/duration_image.png'
img = Image.open(file_path)

# Increase the image display size
plt.figure(figsize=(15,6))  # You can adjust this size as needed

# Display the image
plt.imshow(img)
plt.axis('off')  # Hides the axis for a cleaner display
plt.show()


### Granite Time Series Model

In [ ]:
# Install the tsfm library
! pip install "tsfm_public[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.8" -q

In [ ]:
### Dataset is "amf-handover-request-durations"

In [ ]:
import math
import os
import pandas as pd
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed

from tsfm_public import (
    TimeSeriesPreprocessor,
    TinyTimeMixerForPrediction,
    TrackingCallback,
    count_parameters,
    get_datasets,
)
from tsfm_public.toolkit.visualization import plot_predictions

# Set seed for reproducibility
SEED = 42
set_seed(SEED)

# Path to your dataset
file_path = 'open-digital-platform-2_0/5G_Emulator_API/core_network/amf-handover-request-durations.csv'

# Load the dataset into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows
df.head()

# Results directory
OUT_DIR = "ttm_finetuned_models/"
os.makedirs(OUT_DIR, exist_ok=True)

# TTM model branch
# Use main for 512-96 model
# Use "1024_96_v1" for 1024-96 model
TTM_MODEL_REVISION = "main"

# Forecasting parameters
context_length = 512
forecast_length = 96
fewshot_fraction = 0.05


### Inspect AMF Handover Dataset and preprocess

In [ ]:
# Inspect the first few rows of the dataset to understand its structure
df.head()

# Display the columns of the dataset
df.columns

# Depending on your task, identify the target and time columns
# Let's assume 'handover_duration' is your target and 'timestamp' is your time column

# Convert the 'timestamp' column to datetime format if it's not already
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Sort by timestamp to ensure temporal order
df = df.sort_values(by='Timestamp')

# Set the 'timestamp' as the index
df.set_index('Timestamp', inplace=True)

# Display the DataFrame after preprocessing
df.head()


In [ ]:
import matplotlib.pyplot as plt

# Customize the plot
plt.figure(figsize=(20, 5))
plt.plot(df.iloc[:1000].index, df.iloc[:1000]["Cumulative_Sum_Value"], color='purple', linestyle='--', linewidth=2, marker='o', markerfacecolor='orange', markersize=5)

# Adding title and labels
plt.title("Time in seconds spend on Handover Operations via AMF CNF", fontsize=16)
plt.xlabel("Timestamp", fontsize=14)
plt.ylabel("Cumulative Sum Value", fontsize=14)

# Add grid lines
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Display the plot
plt.show()


### All in one Granite Time Series Zero Shot on AMF Handover Duration Time Series Data from AMF CNF

In [ ]:
!pip install pandas -q
!pip install torch -q
!pip install transformers -q
!pip install matplotlib -q

In [ ]:
import os
import pandas as pd
import torch
from transformers import set_seed
import numpy as np
import matplotlib.pyplot as plt

# Import necessary modules from tsfm_public
from tsfm_public import (
    TinyTimeMixerForPrediction,
    count_parameters,
)

# Set seed for reproducibility
SEED = 42
set_seed(SEED)

# Update the file_path based on the provided location
file_path = 'open-digital-platform-2_0/5G_Emulator_API/core_network/amf-handover-request-durations.csv'

# Step 1: Load the Dataset
try:
    df = pd.read_csv(file_path, parse_dates=['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    print("DataFrame head:")
    print(df.head())
    print("\nDataFrame columns:")
    print(df.columns)
    print(f"\nTotal number of rows in the DataFrame: {len(df)}\n")
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please check the file path and ensure the file is uploaded correctly.")
    exit(1)

# Results directory
OUT_DIR = "ttm_finetuned_models/"
os.makedirs(OUT_DIR, exist_ok=True)

# TTM model branch
TTM_MODEL_REVISION = "main"

# Set context_length and forecast_length to match the model's configuration
context_length = 512  # Model's expected context length
forecast_length = 96  # Model's expected forecast length

print(f"Context length: {context_length}")
print(f"Forecast length: {forecast_length}\n")

# Reset the index to make 'Timestamp' a column again
df = df.reset_index()

# Ensure that the 'Timestamp' column is of datetime type
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extract values
values = df['Cumulative_Sum_Value'].values.flatten()

# Prepare the dataset
test_dataset = []

# Number of samples we can create
num_samples = len(values) - forecast_length

for i in range(num_samples):
    # Initialize arrays with padding
    past_values = np.zeros((context_length, 1), dtype=np.float32)  # Add channel dimension
    past_observed_mask = np.zeros((context_length, 1), dtype=bool)  # Add channel dimension

    # Determine the actual length of past values we have
    end_idx = i + 1
    start_idx = max(0, end_idx - context_length)
    actual_length = end_idx - start_idx

    # Fill in the available past values
    past_values[-actual_length:, 0] = values[start_idx:end_idx]
    past_observed_mask[-actual_length:, 0] = True

    # Get future values
    future_values = values[end_idx:end_idx+forecast_length]
    future_observed_mask = np.ones((forecast_length, 1), dtype=bool)  # Add channel dimension

    # Skip samples where future values are incomplete
    if len(future_values) < forecast_length:
        continue

    sample = {
        'past_values': torch.tensor(past_values, dtype=torch.float32),
        'future_values': torch.tensor(future_values, dtype=torch.float32),
        'past_observed_mask': torch.tensor(past_observed_mask, dtype=torch.bool),
        'future_observed_mask': torch.tensor(future_observed_mask, dtype=torch.bool),
    }
    test_dataset.append(sample)

# Display dataset information
print(f"Data lengths: test = {len(test_dataset)}\n")

# Optional: Inspect the first item to ensure correct format
print("First item in the test dataset:")
for key, value in test_dataset[0].items():
    print(f"{key}: {value.shape}")
print()

# Step 4: Load the Pre-trained TinyTimeMixerForPrediction Model
zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-v1",
    revision=TTM_MODEL_REVISION
)

# Display information about the loaded model
print("Loaded model:")
print(zeroshot_model)
print(f"Number of parameters: {count_parameters(zeroshot_model)}\n")

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
zeroshot_model.to(device)
zeroshot_model.eval()

# Step 5: Visualize Results

# Choose a few samples to plot
num_samples_to_plot = 5
indices = np.random.choice(len(test_dataset), size=num_samples_to_plot, replace=False)
samples = [test_dataset[i] for i in indices]

# Prepare inputs for the model
batch_past_values = torch.stack([s['past_values'] for s in samples]).to(device)
batch_observed_mask = torch.stack([s['past_observed_mask'] for s in samples]).to(device)

# Call the model
with torch.no_grad():
    output = zeroshot_model(
        past_values=batch_past_values,
        observed_mask=batch_observed_mask,
    )

# Get predictions
predictions = output.prediction_outputs.squeeze(-1).cpu().numpy()

# For each sample, plot the past_values, true future_values, and predicted future_values
for i, s in enumerate(samples):
    past_values = s['past_values'].squeeze(-1).cpu().numpy()
    future_values = s['future_values'].cpu().numpy()
    pred_values = predictions[i]

    # Create time indices
    past_time = np.arange(len(past_values))
    future_time = np.arange(len(past_values), len(past_values) + len(future_values))

    # Combine past and future times and values for plotting
    total_time = np.concatenate((past_time, future_time))
    total_values = np.concatenate((past_values, future_values))

    # Customize the plot
    plt.figure(figsize=(20, 5))
    plt.plot(total_time, total_values, color='purple', linestyle='--', linewidth=2, marker='o',
             markerfacecolor='orange', markersize=5, label='Actual Values')
    plt.plot(future_time, pred_values, color='green', linestyle='-', linewidth=2, label='Predicted Future Values')

    # Adding title and labels
    plt.title("Time in seconds spent on Handover Operations via AMF CNF", fontsize=16)
    plt.xlabel("Time Steps", fontsize=14)
    plt.ylabel("Cumulative Sum Value", fontsize=14)

    # Add grid lines
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Show legend
    plt.legend()

    # Display the plot
    plt.show()
